In [77]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    memory_key="history",  # THIS IS IMPORTANT PART GIVE HISTORY MEM KEY
    return_messages=True
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}")
    ]
)


def load_memory(input):
    return memory.load_memory_variables({})["history"]


# RunnablePassthrough.assign -> load_memory() return X -> prompt의 인자 history에 X 할당
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content}
    )
    print(result)


invoke_chain("My name is Seonghwan")

content='Hello Seonghwan! How can I assist you today?'
